In [1]:
import sys

In [2]:
import intake
import pandas as pd
import geopandas as gpd
import numpy as np
from calitp_data_analysis import geography_utils
from siuba import *
import gtfs_segments
import shapely

In [45]:
from shared_utils import catalog_utils, rt_dates, rt_utils
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

In [4]:
from segment_speed_utils import helpers

In [5]:
analysis_date = rt_dates.DATES['feb2025']

# Exploratory Analysis

Start working on how to identify and count "border zones" where transit runs along Census Tract boundaries and VRH/VRM should be split accordingly (avoiding double-counting or arbitrary allocation to only one border tract)

## trip - linestring interpolation

could use this again? https://github.com/cal-itp/data-analyses/blob/530f2d5cf4419a2403d6485845d91ac4bc65e672/_shared_utils/shared_utils/rt_utils.py#L574-L579

https://github.com/cal-itp/data-analyses/blob/530f2d5cf4419a2403d6485845d91ac4bc65e672/rt_delay/rt_analysis/rt_parser.py#L90

* fillna both directions arrival <-> departure time
* then dropna

## Projected st via `stop_times_direction`

### is this accurate?

In [6]:
path = f'{GTFS_DATA_DICT.rt_vs_schedule_tables.dir}{GTFS_DATA_DICT.rt_vs_schedule_tables.stop_times_direction}_{analysis_date}.parquet'
# ST_DIR_COLS = ['trip_instance_key', 'stop_sequence', 'stop_meters', 'stop_id', 'geometry']
st_dir = gpd.read_parquet(path)

In [7]:
test = st_dir.query('trip_instance_key == "98c8b779600cc0c399755929110a83c4"').sort_values('stop_sequence')

In [8]:
test.crs

<Projected CRS: EPSG:3310>
Name: NAD83 / California Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - California.
- bounds: (-124.45, 32.53, -114.12, 42.01)
Coordinate Operation:
- name: California Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [11]:
shapes_proj = shapes.to_crs(test.crs)

In [12]:
test_geom = shapes_proj.query('shape_array_key == "3caab5c44277cbdc8fbc755bc0ea7633"').geometry.iloc[0]

In [13]:
test_geom.project(test.geometry.iloc[0])

0.0

In [14]:
test

,feed_key,stop_id,stop_sequence,schedule_gtfs_dataset_key,trip_instance_key,shape_array_key,stop_name,geometry,stop_meters,prior_stop_sequence,subseq_stop_sequence,stop_primary_direction,stop_pair,stop_pair_name
2586785,e47b82448b027d26835d46f2f7cbdc1c,51636,1,c499f905e33929a641f083dad55c521e,98c8b779600cc0c399755929110a83c4,3caab5c44277cbdc8fbc755bc0ea7633,10th St & Washington St,POINT (-199973.418 -21403.301),0.000000,NaN,2.0,Unknown,51636__59444,10th St & Washington St__11th St & Clay St
2586786,e47b82448b027d26835d46f2f7cbdc1c,59444,2,c499f905e33929a641f083dad55c521e,98c8b779600cc0c399755929110a83c4,3caab5c44277cbdc8fbc755bc0ea7633,11th St & Clay St,POINT (-199972.230 -21324.489),215.765913,1.0,3.0,Northbound,59444__53108,11th St & Clay St__Broadway & 12th St
2586787,e47b82448b027d26835d46f2f7cbdc1c,53108,3,c499f905e33929a641f083dad55c521e,98c8b779600cc0c399755929110a83c4,3caab5c44277cbdc8fbc755bc0ea7633,Broadway & 12th St,POINT (-199734.943 -21303.047),537.650981,2.0,4.0,Eastbound,53108__52894,Broadway & 12th St__Broadway & 15th St
2586788,e47b82448b027d26835d46f2f7cbdc1c,52894,4,c499f905e33929a641f083dad55c521e,98c8b779600cc0c399755929110a83c4,3caab5c44277cbdc8fbc755bc0ea7633,Broadway & 15th St,POINT (-199626.961 -21102.141),768.695450,3.0,5.0,Northbound,52894__53335,Broadway & 15th St__Broadway & 17th St
2586789,e47b82448b027d26835d46f2f7cbdc1c,53335,5,c499f905e33929a641f083dad55c521e,98c8b779600cc0c399755929110a83c4,3caab5c44277cbdc8fbc755bc0ea7633,Broadway & 17th St,POINT (-199544.907 -20940.755),952.019849,4.0,6.0,Northbound,53335__57999,Broadway & 17th St__Uptown Transit Center
2586790,e47b82448b027d26835d46f2f7cbdc1c,57999,6,c499f905e33929a641f083dad55c521e,98c8b779600cc0c399755929110a83c4,3caab5c44277cbdc8fbc755bc0ea7633,Uptown Transit Center,POINT (-199510.604 -20598.973),1384.352678,5.0,7.0,Northbound,57999__59991,Uptown Transit Center__Telegraph Av & 24th St
2586791,e47b82448b027d26835d46f2f7cbdc1c,59991,7,c499f905e33929a641f083dad55c521e,98c8b779600cc0c399755929110a83c4,3caab5c44277cbdc8fbc755bc0ea7633,Telegraph Av & 24th St,POINT (-199405.895 -20100.640),1896.443121,6.0,8.0,Northbound,59991__51634,Telegraph Av & 24th St__Telegraph Av & 27th St
2586792,e47b82448b027d26835d46f2f7cbdc1c,51634,8,c499f905e33929a641f083dad55c521e,98c8b779600cc0c399755929110a83c4,3caab5c44277cbdc8fbc755bc0ea7633,Telegraph Av & 27th St,POINT (-199342.007 -19830.511),2175.848472,7.0,9.0,Northbound,51634__53036,Telegraph Av & 27th St__Telegraph Av & 30th St
2586793,e47b82448b027d26835d46f2f7cbdc1c,53036,9,c499f905e33929a641f083dad55c521e,98c8b779600cc0c399755929110a83c4,3caab5c44277cbdc8fbc755bc0ea7633,Telegraph Av & 30th St,POINT (-199266.491 -19485.822),2530.389449,8.0,10.0,Northbound,53036__53038,Telegraph Av & 30th St__Telegraph Av & 34th St
2586794,e47b82448b027d26835d46f2f7cbdc1c,53038,10,c499f905e33929a641f083dad55c521e,98c8b779600cc0c399755929110a83c4,3caab5c44277cbdc8fbc755bc0ea7633,Telegraph Av & 34th St,POINT (-199187.387 -19156.341),2870.206166,9.0,11.0,Northbound,53038__55375,Telegraph Av & 34th St__Telegraph Av & 38th St


### functions

In [15]:
def attach_projected_stop_times(analysis_date: str):
    '''
    
    '''
    path = f'{GTFS_DATA_DICT.rt_vs_schedule_tables.dir}{GTFS_DATA_DICT.rt_vs_schedule_tables.stop_times_direction}_{analysis_date}.parquet'
    ST_DIR_COLS = ['trip_instance_key', 'stop_sequence', 'stop_meters', 'stop_id']
    st_dir = gpd.read_parquet(path)[ST_DIR_COLS]
    st = helpers.import_scheduled_stop_times(analysis_date, get_pandas=True)
    trips = helpers.import_scheduled_trips(analysis_date, columns=['trip_id', 'trip_instance_key', 'feed_key',
                                                                  'shape_array_key'])
    st = st.merge(trips, on = ['feed_key', 'trip_id'])
    return st.merge(st_dir, on = ['trip_instance_key', 'stop_sequence', 'stop_id'])

In [17]:
st_proj = attach_projected_stop_times(analysis_date)

In [18]:
st_proj.arrival_sec.isna().value_counts()

False    4232118
True       90428
Name: arrival_sec, dtype: int64

In [19]:
st_proj.departure_sec.isna().value_counts()

False    4232118
True       90428
Name: departure_sec, dtype: int64

In [133]:
# st_proj.query('departure_sec.isna() & arrival_sec.isna()') # same subset, no point in fillna

In [22]:
st_proj.to_parquet(f'st_proj_{analysis_date}.parquet')

## projecting TSI interpolation points

In [60]:
shapes = helpers.import_scheduled_shapes(analysis_date, crs=geography_utils.CA_NAD83Albers_m)

In [141]:
def read_tracts_borders(analysis_date, shapes):
    test_tract_borders = gpd.read_parquet(f'test_tracts_borders_{analysis_date}.parquet')
    test_tract_borders = (test_tract_borders.drop(columns=['geometry'])
                     .assign(tsi_segment_id = test_tract_borders.tract.combine_first(test_tract_borders.intersection_hash))
                     )

    test_tract_borders = (shapes.merge(test_tract_borders, on='shape_array_key')
                         .rename(columns={'geometry': 'shape_geometry'}))

    test_tract_borders = test_tract_borders.assign(
        start_meters = test_tract_borders.shape_geometry.project(test_tract_borders.start)
    )
    test_tract_borders = test_tract_borders.sort_values('start_meters').reset_index(drop=True)
    return test_tract_borders

In [142]:
test_tract_borders = read_tracts_borders(analysis_date, shapes)

## interpolation with arrays

### try one trip

In [183]:
%%time

one_trip = st_proj.query('trip_instance_key == "98c8b779600cc0c399755929110a83c4"').sort_values('stop_sequence')

CPU times: user 132 ms, sys: 163 μs, total: 132 ms
Wall time: 130 ms


In [127]:
one_trip.head(3)

,feed_key,feed_timezone,base64_url,trip_id,stop_id,stop_sequence,timepoint,arrival_sec,departure_sec,arrival_hour,departure_hour,trip_instance_key,shape_array_key,stop_meters
2812526,e47b82448b027d26835d46f2f7cbdc1c,US/Pacific,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,7071020,51636,1,1.0,47160.0,47160.0,13.0,13.0,98c8b779600cc0c399755929110a83c4,3caab5c44277cbdc8fbc755bc0ea7633,0.000000
2812543,e47b82448b027d26835d46f2f7cbdc1c,US/Pacific,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,7071020,59444,2,0.0,47230.0,47230.0,13.0,13.0,98c8b779600cc0c399755929110a83c4,3caab5c44277cbdc8fbc755bc0ea7633,215.765913
2812555,e47b82448b027d26835d46f2f7cbdc1c,US/Pacific,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,7071020,53108,3,0.0,47329.0,47329.0,13.0,13.0,98c8b779600cc0c399755929110a83c4,3caab5c44277cbdc8fbc755bc0ea7633,537.650981


In [128]:
shape_array = one_trip.stop_meters.to_numpy()
dt_float_array = one_trip.arrival_sec.to_numpy()

In [130]:
test_tract_borders['arrival_sec'] = test_tract_borders.start_meters.apply(
    rt_utils.time_at_position_numba, shape_array=shape_array, dt_float_array = dt_float_array)
test_tract_borders = test_tract_borders.assign(arrival_sec_next = test_tract_borders.arrival_sec.shift(-1),
                                               trip_instance_key = one_trip.trip_instance_key.iloc[0])
test_tract_borders.loc[0,'arrival_sec'] = one_trip.arrival_sec.min()
test_tract_borders.loc[test_tract_borders.index.max(),'arrival_sec_next'] = one_trip.arrival_sec.max()
test_tract_borders = test_tract_borders.assign(segment_seconds = test_tract_borders.arrival_sec_next - test_tract_borders.arrival_sec)

### create function -- with `test_tract_borders`

In [135]:
test_tract_borders.columns

Index(['shape_array_key', 'shape_geometry', 'tract', 'tract_1', 'tract_2',
       'intersection_hash', 'border', 'start', 'end', 'tsi_segment_id',
       'start_meters', 'arrival_sec', 'arrival_sec_next', 'trip_instance_key',
       'segment_seconds'],
      dtype='object')

In [136]:
tracts_borders_cols = ['shape_array_key', 'tsi_segment_id', 'start_meters']

In [151]:
tracts_borders = read_tracts_borders(analysis_date, shapes)[tracts_borders_cols]

In [152]:
tracts_borders_trips = tracts_borders.merge(st_proj[['shape_array_key', 'trip_instance_key']].drop_duplicates(), on='shape_array_key')

In [153]:
tracts_borders_trips

,shape_array_key,tsi_segment_id,start_meters,trip_instance_key
0,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.000000,98c8b779600cc0c399755929110a83c4
1,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.000000,d7a924d5a3701b8e501960eaa3f75fd3
2,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.000000,95d92c621f97a404f74e94b16ce3f958
3,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.000000,5e00add8b1104ca5f06d8ecf26a51ddd
4,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.000000,71cde235265cf3101d168f7374acdc71
...,...,...,...,...
1371,3caab5c44277cbdc8fbc755bc0ea7633,-5279734711215237120.0,8660.491989,12204f1b76f3daeb84e79760aeb431b3
1372,3caab5c44277cbdc8fbc755bc0ea7633,-5279734711215237120.0,8660.491989,96ad1d5a8a96baf6adc6e81c33fb6792
1373,3caab5c44277cbdc8fbc755bc0ea7633,-5279734711215237120.0,8660.491989,239e0a0774af5c51508bf0ac0aec4518
1374,3caab5c44277cbdc8fbc755bc0ea7633,-5279734711215237120.0,8660.491989,bf1b27f19b91b5e5c4f6bdf6d67af31e


In [161]:
def tract_border_time_by_trip(tracts_borders_trip_df: pd.DataFrame, st_proj_df: pd.DataFrame):
    '''
    '''
    
    one_trip = st_proj_df.query('trip_instance_key == @tracts_borders_trip_df.trip_instance_key.iloc[0]').sort_values('stop_sequence')
    shape_array = one_trip.stop_meters.to_numpy()
    dt_float_array = one_trip.arrival_sec.to_numpy()
    tracts_borders_trip_df['arrival_sec'] = tracts_borders_trip_df.start_meters.apply(
        rt_utils.time_at_position_numba, shape_array=shape_array, dt_float_array = dt_float_array)
    tracts_borders_trip_df = tracts_borders_trip_df.assign(arrival_sec_next = tracts_borders_trip_df.arrival_sec.shift(-1),
                                                   trip_instance_key = one_trip.trip_instance_key.iloc[0])
    tracts_borders_trip_df.loc[tracts_borders_trip_df.index.min(),'arrival_sec'] = one_trip.arrival_sec.min()
    tracts_borders_trip_df.loc[tracts_borders_trip_df.index.max(),'arrival_sec_next'] = one_trip.arrival_sec.max()
    tracts_borders_trip_df = tracts_borders_trip_df.assign(segment_seconds = tracts_borders_trip_df.arrival_sec_next - tracts_borders_trip_df.arrival_sec)
    
    return tracts_borders_trip_df

In [165]:
all_trip_test = tracts_borders_trips.groupby('trip_instance_key', group_keys=False).apply(tract_border_time_by_trip, st_proj_df = st_proj)

In [166]:
all_trip_test

,shape_array_key,tsi_segment_id,start_meters,trip_instance_key,arrival_sec,arrival_sec_next,segment_seconds
0,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.000000,98c8b779600cc0c399755929110a83c4,47160.000000,47275.513869,115.513869
1,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.000000,d7a924d5a3701b8e501960eaa3f75fd3,28440.000000,28538.617828,98.617828
2,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.000000,95d92c621f97a404f74e94b16ce3f958,33480.000000,33578.617828,98.617828
3,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.000000,5e00add8b1104ca5f06d8ecf26a51ddd,78000.000000,78065.745219,65.745219
4,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.000000,71cde235265cf3101d168f7374acdc71,30600.000000,30698.617828,98.617828
...,...,...,...,...,...,...,...
1371,3caab5c44277cbdc8fbc755bc0ea7633,-5279734711215237120.0,8660.491989,12204f1b76f3daeb84e79760aeb431b3,50751.259581,50880.000000,128.740419
1372,3caab5c44277cbdc8fbc755bc0ea7633,-5279734711215237120.0,8660.491989,96ad1d5a8a96baf6adc6e81c33fb6792,45711.259581,45840.000000,128.740419
1373,3caab5c44277cbdc8fbc755bc0ea7633,-5279734711215237120.0,8660.491989,239e0a0774af5c51508bf0ac0aec4518,73812.593708,73920.000000,107.406292
1374,3caab5c44277cbdc8fbc755bc0ea7633,-5279734711215237120.0,8660.491989,bf1b27f19b91b5e5c4f6bdf6d67af31e,34071.259581,34200.000000,128.740419


In [167]:
all_trip_test.query('trip_instance_key == "0d9b991fd66489617e4b841440a3ad6d"')

,shape_array_key,tsi_segment_id,start_meters,trip_instance_key,arrival_sec,arrival_sec_next,segment_seconds
28,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.000000,0d9b991fd66489617e4b841440a3ad6d,63720.000000,63835.513869,115.513869
114,3caab5c44277cbdc8fbc755bc0ea7633,-3476482707806839296.0,363.748082,0d9b991fd66489617e4b841440a3ad6d,63835.513869,63917.689256,82.175387
200,3caab5c44277cbdc8fbc755bc0ea7633,8444362949197194240.0,639.627811,0d9b991fd66489617e4b841440a3ad6d,63917.689256,64112.851381,195.162124
286,3caab5c44277cbdc8fbc755bc0ea7633,06001402800,1294.755431,0d9b991fd66489617e4b841440a3ad6d,64112.851381,64206.157862,93.306482
372,3caab5c44277cbdc8fbc755bc0ea7633,06001401300,1651.114954,0d9b991fd66489617e4b841440a3ad6d,64206.157862,64546.328441,340.170579
458,3caab5c44277cbdc8fbc755bc0ea7633,06001401100,3008.454903,0d9b991fd66489617e4b841440a3ad6d,64546.328441,64949.654322,403.325881
544,3caab5c44277cbdc8fbc755bc0ea7633,-8684109331624108032.0,4510.932927,0d9b991fd66489617e4b841440a3ad6d,64949.654322,64987.237866,37.583543
630,3caab5c44277cbdc8fbc755bc0ea7633,06001400300,4643.226772,0d9b991fd66489617e4b841440a3ad6d,64987.237866,65059.846661,72.608796
716,3caab5c44277cbdc8fbc755bc0ea7633,1701263025624594432.0,4972.931258,0d9b991fd66489617e4b841440a3ad6d,65059.846661,65123.798193,63.951532
802,3caab5c44277cbdc8fbc755bc0ea7633,9114761875024974848.0,5381.856488,0d9b991fd66489617e4b841440a3ad6d,65123.798193,65281.990288,158.192095


In [180]:
all_trip_test.query('tsi_segment_id == "06001403100" & arrival_sec > 60000').sort_values('segment_seconds')

,shape_array_key,tsi_segment_id,start_meters,trip_instance_key,arrival_sec,arrival_sec_next,segment_seconds
3,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.0,5e00add8b1104ca5f06d8ecf26a51ddd,78000.0,78065.745219,65.745219
45,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.0,b32dd6d1595e58cdb5846918ccf4e4a2,82800.0,82865.745219,65.745219
29,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.0,acc50431e96cb57f74fe74325688f8f8,79200.0,79265.745219,65.745219
24,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.0,5e0fadab0370430f0656c04e91645a47,85200.0,85265.745219,65.745219
19,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.0,facc2913553362bd637f0c7be8970221,81600.0,81665.745219,65.745219
16,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.0,5618e369e6ed8786c78bbef18eb0abe6,80400.0,80465.745219,65.745219
55,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.0,448f22153e85a552a0bfe2faad5e8745,86400.0,86465.745219,65.745219
61,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.0,d4c25f6c67dcfd9911a00e65ede4c5a9,76800.0,76865.745219,65.745219
10,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.0,b5f93d259f8d7917f0f7b9170d23796c,87600.0,87665.745219,65.745219
6,3caab5c44277cbdc8fbc755bc0ea7633,06001403100,0.0,3f98f4f6995c637d3f47e6a38afdb3b9,88800.0,88865.745219,65.745219


In [170]:
import datetime as dt

In [182]:
str(dt.timedelta(seconds=72082.181523))

'20:01:22.181523'